## Setup

In [1]:
# Change the data name
import os

data_root = "clothes/train"
categories = ["pants", "shirts", "shorts", "t-shirts"]

for category in categories:
    category_folder = os.path.join(data_root, category)
    
    for fname in os.listdir(category_folder):
        if fname.endswith('.jpg') and fname.startswith(category + "_"):
            parts = fname.split('_')
            if len(parts) >= 2:
                number = parts[1].split('.')[0]  # Extract number without .jpg
                new_name = f"{number}_{category}.jpg"
                old_path = os.path.join(category_folder, fname)
                new_path = os.path.join(category_folder, new_name)
                
                print(f"Renaming {old_path} -> {new_path}")
                os.rename(old_path, new_path)

print("Done renaming all images!")


Done renaming all images!


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())

2.0.1
CUDA available: False


## Model 1

In [3]:
# import necessary packages
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# Define paths
data_root = "clothes/train"
metadata_path = "metadata.json"
categories = ["pants", "shirts", "shorts", "t-shirts"]
model_save_path = "model_1.pth"

In [5]:
# Load metadata
with open(metadata_path, 'r') as f:
    metadata = json.load(f)


In [11]:
# Create a combined dataset from all categories
class ClothingDataset(Dataset):
    def __init__(self, metadata, categories, data_root, transform=None):
        self.data_root = data_root
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Map category names to numerical labels
        self.category_to_idx = {cat: idx for idx, cat in enumerate(categories)}
        
        # Collect all images and their labels
        for category in categories:
            category_items = metadata.get(category, [])
            for item in category_items:
                img_path = os.path.join(data_root, category, item['filename'])
                if os.path.exists(img_path):
                    self.images.append(img_path)
                    self.labels.append(self.category_to_idx[category])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Load and transform image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [12]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [13]:
# Create dataset and split into train/validation
full_dataset = ClothingDataset(metadata, categories, data_root, transform)
train_indices, val_indices = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)

train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset, val_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Define CNN model (using ResNet-18 pretrained)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  # correct way for torchvision>=0.13
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(categories))  # 4 output classes

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 124MB/s]


In [14]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')

# Save the trained model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/10, Loss: 1.0614, Validation Accuracy: 87.50%
Epoch 2/10, Loss: 0.0373, Validation Accuracy: 93.75%
Epoch 3/10, Loss: 0.0298, Validation Accuracy: 93.75%
Epoch 4/10, Loss: 0.0432, Validation Accuracy: 93.75%
Epoch 5/10, Loss: 0.0227, Validation Accuracy: 100.00%
Epoch 6/10, Loss: 0.0003, Validation Accuracy: 93.75%
Epoch 7/10, Loss: 0.0117, Validation Accuracy: 87.50%
Epoch 8/10, Loss: 0.0028, Validation Accuracy: 62.50%
Epoch 9/10, Loss: 0.0009, Validation Accuracy: 56.25%
Epoch 10/10, Loss: 0.0010, Validation Accuracy: 62.50%
Model saved to model_1.pth


In [ ]:
!conda install -c conda-forge faiss-cpu -y

Retrieving notices: done


In [17]:
import os
import json
import faiss
import numpy as np
from tqdm import tqdm

data_root = "clothes/train"
categories = ["pants", "shirts", "shorts", "t-shirts"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Identity()  # Remove classifier
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

embeddings = []
metadata = []

# Extract embeddings
with torch.no_grad():
    for category in categories:
        folder = os.path.join(data_root, category)
        for fname in tqdm(os.listdir(folder), desc=f"Processing {category}"):
            if fname.endswith('.jpg'):
                img_path = os.path.join(folder, fname)
                image = Image.open(img_path).convert('RGB')
                image = transform(image).unsqueeze(0).to(device)

                emb = model(image).cpu().numpy().flatten()
                embeddings.append(emb.tolist())

                metadata.append({
                    "image_path": img_path,
                    "category": category
                })

# Save embeddings to json
with open('clothing_embedding.json', 'w') as f:
    json.dump(metadata, f, indent=2)

# Build FAISS index
embeddings_np = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

faiss.write_index(index, 'clothing_faiss.index')

print("Done! Files saved:")
print("- clothing_embedding.json")
print("- clothing_faiss.index")


ModuleNotFoundError: No module named 'faiss'